In [2]:
import pandas as pd

In [13]:
ios = pd.read_csv('app_reviews/rv_ios_app_reviews.csv')
ios['Content'] = ios['label_title']+ios['review']
ios = ios.drop(['app_name','app_version','label_title','review'], axis=1)
print(ios.shape)
ios.head()

(1962, 2)


,store_rating,Content
0,1,Does nothingWill not do anything without paid ...
1,1,Don’t receive confirmation emailTried several ...
2,1,This is not ready for release!!! Avoid!!!!To s...
3,1,"GPS directions badFirst time using RV Life, it..."
4,5,Outstanding ! Essential for campers!This is th...


In [18]:
ios['store_rating'].value_counts()

5    1157
1     298
4     283
2     114
3     110
Name: store_rating, dtype: int64

In [14]:
android = pd.read_csv('app_reviews/rv_android_reviews.csv')
# android = android.drop(['Unnamed: 0','Unnamed: 0.1','Date','App_name'], axis=1)
# android = android.rename(columns={'Numbers':'store_rating'})
print(android.shape)
android.head()

(639, 2)


,Content,store_rating
0,Worked fine for a month. Then when it came tim...,1
1,Still navigating through 7.5t restrictions reg...,2
2,Ever since the update I am unable to pay for t...,3
3,OK app but a bit unpredictable with routings. ...,4
4,Loved using this app for trucking. Since the r...,5


In [4]:
android = pd.read_csv('app_reviews/rv_android_reviews.csv')
# android = android.drop(['Unnamed: 0','Unnamed: 0.1','Date','App_name'], axis=1)
# android = android.rename(columns={'Numbers':'store_rating'})
print(android.shape)
android.head()

(6746, 6)


,Unnamed: 0,Content,Star,Number,Date,App_name
0,0,"The potential is there, but some serious work ...",2,1,"August 17, 2019",rvlife.rvlife
1,1,"licencing problems, working with their support...",2,2,"August 2, 2019",rvlife.rvlife
2,2,not impressed. voice is not loud enough. print...,3,3,"August 7, 2019",rvlife.rvlife
3,3,Ver 2.0.10. The licensing issue has finally be...,1,4,"August 9, 2019",rvlife.rvlife
4,4,Does not allow for waypoints or route deviatio...,2,5,"August 18, 2019",rvlife.rvlife


In [5]:
android['Star'].value_counts()

5    4295
4    1385
1     507
3     336
2     223
Name: Star, dtype: int64

In [15]:
df = pd.concat([android, ios], join='outer', ignore_index=True, axis=0, sort=True)
print(df.shape)
df.head()

(2601, 2)


,Content,store_rating
0,Worked fine for a month. Then when it came tim...,1
1,Still navigating through 7.5t restrictions reg...,2
2,Ever since the update I am unable to pay for t...,3
3,OK app but a bit unpredictable with routings. ...,4
4,Loved using this app for trucking. Since the r...,5


In [16]:
df.isnull().sum()

Content         0
store_rating    0
dtype: int64

In [ ]:
df['store_rating'].value_counts()

In [ ]:
df['rating'] = df['store_rating'].replace({4:5, 2:1, 3:1}).astype(float).astype(str)
# df['rating'] = df['store_rating']
# df['rating'] = df['rating']
df['rating'].value_counts()

In [23]:
df.dtypes

Content         object
store_rating     int64
rating          object
dtype: object

In [24]:
import spacy
import scattertext
import pandas as pd
pd.set_option('display.max_columns', 500) # Make sure we can see all of the columns
pd.set_option('display.max_rows', 200)

In [ ]:
nlp = spacy.load("en_core_web_lg")

In [112]:
# add stop words
with open('stopwords.txt', 'r') as f:
    str = f.read()
    set_stopwords = set(str.split('\n'))
nlp.Defaults.stop_words |= set_stopwords

In [ ]:
corpus = (scattertext.CorpusFromPandas(Shopping_yelp_sample, 
                                      category_col='rating', 
                                      text_col='Content',
                                      nlp=nlp)
          .build()
          .remove_terms(nlp.Defaults.stop_words, ignore_absences=True)
         )

In [ ]:
term_freq_df = corpus.get_term_freq_df()
term_freq_df['highratingscore'] = corpus.get_scaled_f_scores('5.0')
term_freq_df['poorratingscore'] = corpus.get_scaled_f_scores('1.0')

df_high = term_freq_df.sort_values(by='highratingscore', 
                                   ascending = False)
df_poor = term_freq_df.sort_values(by='poorratingscore', 
                                   ascending=False)

# df_high = df_high[['highratingscore', 'poorratingscore']]
df_high['highratingscore'] = round(df_high['highratingscore'], 2)
df_high['poorratingscore'] = round(df_high['poorratingscore'], 2)
df_high = df_high.reset_index(drop=False)
# df_high = df_high.head(20)

# df_poor = df_poor[['highratingscore', 'poorratingscore']]
df_poor['highratingscore'] = round(df_poor['highratingscore'], 2)
df_poor['poorratingscore'] = round(df_poor['poorratingscore'], 2)
df_poor = df_poor.reset_index(drop=False)
# df_poor = df_poor.head(20)

# df_terms = pd.concat([df_high, df_poor],
#                      ignore_index=True)
# df_terms

In [ ]:
Shopping_yelp_sample_high = df_high
Shopping_yelp_sample_poor = df_poor

In [119]:
Shopping_yelp_sample_high.sort_values(by='5.0 freq', ascending = False).head(100)

,term,5.0 freq,1.0 freq,highratingscore,poorratingscore
1245604,store,18755,17941,0.12,0.88
1244761,service,15985,13303,0.14,0.86
2483,good,12253,6143,0.91,0.09
2685,new,11188,5889,0.90,0.10
453,friendly,10763,1819,0.97,0.03
1282,staff,10484,3434,0.94,0.06
3255,and the,10252,6134,0.89,0.11
558,best,9815,1912,0.97,0.03
406,helpful,9431,1463,0.97,0.03
452,recommend,9158,1543,0.97,0.03


In [118]:
Shopping_yelp_sample_poor.sort_values(by='1.0 freq', ascending = False).head(100)

,term,5.0 freq,1.0 freq,highratingscore,poorratingscore
3886,store,18755,17941,0.12,0.88
4729,service,15985,13303,0.14,0.86
2437,customer,8848,11209,0.09,0.91
408,said,3272,11054,0.03,0.97
1717,$,7074,10771,0.08,0.92
3026,to the,7544,8476,0.10,0.90
3856,went,8079,7766,0.12,0.88
2878,customer service,6603,7624,0.10,0.90
2508,the store,5817,7270,0.09,0.91
3244,on the,6500,7039,0.11,0.89


In [94]:
Auto_Repair_yelp_high.sort_values(by='5.0 freq', ascending = False).head(100)

,term,5.0 freq,1.0 freq,highratingscore,poorratingscore
2976,car,40357,41815,0.87,0.13
1849,service,26490,19259,0.91,0.09
1219,work,15144,8105,0.93,0.07
2299,my car,14244,12133,0.89,0.11
2116,new,10707,8581,0.90,0.10
1172,experience,10665,5513,0.94,0.06
1108,shop,9506,4745,0.94,0.06
371,recommend,9271,2003,0.98,0.02
2210,i have,8976,7421,0.89,0.11
3098,of the,8714,9344,0.87,0.13


In [93]:
Auto_Repair_yelp_poor.sort_values(by='1.0 freq', ascending = False).head(100)

,term,5.0 freq,1.0 freq,highratingscore,poorratingscore
881356,car,40357,41815,0.87,0.13
882483,service,26490,19259,0.91,0.09
463,$,5162,16854,0.05,0.95
372,said,3996,14426,0.04,0.96
1747,the car,7244,12467,0.10,0.90
882033,my car,14244,12133,0.89,0.11
2857,vehicle,8222,10800,0.13,0.87
2537,dealership,7487,10499,0.12,0.88
2932,it was,7809,10111,0.13,0.87
2167,oil,6387,9750,0.11,0.89


In [86]:
Hair_Salons_yelp_high.sort_values(by='5.0 freq', ascending = False).head(100)

,term,5.0 freq,1.0 freq,highratingscore,poorratingscore
1502,hair,42750,15626,0.92,0.08
1470,my hair,22150,7970,0.92,0.08
1031,salon,16158,4370,0.94,0.06
1565,cut,15273,5798,0.92,0.08
1171,i have,10864,3286,0.93,0.07
1334,color,9242,3090,0.93,0.07
1763,stylist,8204,3432,0.91,0.09
195,best,7913,571,0.99,0.01
766,i 've,7710,1642,0.95,0.05
626,job,7649,1358,0.96,0.04


In [2]:
Hair_Salons_yelp_poor.sort_values(by='1.0 freq', ascending = False).head(100)

NameError: name 'Hair_Salons_yelp_poor' is not defined

In [ ]:
Fashion_yelp_high.sort_values(by='5.0 freq', ascending = False)

In [ ]:
Fashion_yelp_poor.sort_values(by='1.0 freq', ascending = False).head(100)

In [138]:
Professional_Services_yelp_poor_json = Professional_Services_yelp_poor_json.drop([136,3577,2707,664626,1979,2678])
Professional_Services_yelp_poor_json.sort_values(by='1 freq', ascending = False).head(10)

,term,1 freq,5 freq,highratingscore,poorratingscore
2480,service,8908,7581,0.10,0.90
445,$,5996,1606,0.03,0.97
1022,phone,4776,2213,0.06,0.94
1472,called,4287,2560,0.07,0.93
136,cox,3135,371,0.01,0.99
3577,it was,3005,3433,0.14,0.86
2707,to the,2969,2694,0.11,0.89
664626,work,2900,7039,0.93,0.07
1976,did n't,2855,2069,0.09,0.91
2678,that i,2845,2557,0.11,0.89


In [ ]:
Professional_Services_yelp_poor.sort_values(by='1 freq', ascending = False).head(100)

In [130]:
Professional_Services_yelp_high_json = Professional_Services_yelp_high_json.drop([1777])
Professional_Services_yelp_best_json = Professional_Services_yelp_high_json.sort_values(by='5 freq', ascending = False).head()

,term,1 freq,5 freq,highratingscore,poorratingscore
2721,time,5335,8239,0.89,0.11
302,professional,485,4677,0.98,0.02
2636,new,2438,3861,0.89,0.11
2850,home,2143,3174,0.88,0.12
293,friendly,308,3059,0.98,0.02


In [ ]:
Professional_Services_yelp_high.sort_values(by='5 freq', ascending = False).head()
Professional_Services_yelp_poor_json.sort_values(by='1 freq', ascending = False).head()

In [153]:
df = df.head()
df.to_json('Fashion_yelp_high_rating_words.json', orient='records', lines=True)